In [28]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("cycles").getOrCreate()

In [29]:
# Base path to csv files
base_path = "../raw-files/Person Data/"

# only keep the columns we need for the person data
required_cols = [
    "BusinessEntityID",
    "Title",
    "FirstName",
    "MiddleName",
    "LastName",
    "NameStyle",
    "Demographics",
    "Suffix",
    "EmailAddress",
    "AddressLine1",
    "AddressLine2",
    "PhoneNumber",
]

# Read required files
person_df = spark.read.csv(base_path + "Person Person.csv", header=True)
emailAddress_df = spark.read.csv(base_path + "Person EmailAddress.csv", header=True)
businessEntity_df = spark.read.csv(base_path + "Person BusinessEntity.csv", header=True)
businessEntityAddress_df = spark.read.csv(base_path + "Person BusinessEntityAddress.csv", header=True)
address_df = spark.read.csv(base_path + "Person Address.csv", header=True)
personPhone_df = spark.read.csv(base_path + "Person PersonPhone.csv", header=True)

# todo columns to add: customerkey, geographykey, birthdate,mariatalstatus, gender, yearlyincome, totalchildern, incomegroup, educattion, numberofchildrenatHome, homeownerflag, numberofcarsowned, age, region

person_details_df = (
    person_df.join(emailAddress_df, "BusinessEntityID", "left")
    .join(businessEntity_df, "BusinessEntityID", "left")
    .join(businessEntityAddress_df, "BusinessEntityID", "left")
    .join(address_df, "AddressID", "left")
    .join(personPhone_df, "BusinessEntityID", "left")
)


person_details_df = person_details_df.select(required_cols)

In [30]:
# Base path to csv files
base_path = "../raw-files/Sales Data/"

# only keep the columns we need for the sales data
required_cols = [
    "CustomerKey",
    "BusinessEntityID",
    "Region",
    "SalesOrderNumber",
    "ProductID",
    "OrderQty",
    "UnitPrice",
]

# Read required files
customer_df = spark.read.csv(base_path + "Sales Customer.csv", header=True)
salesOrderHeader_df = spark.read.csv(base_path + "Sales SalesOrderHeader.csv", header=True)
salesOrderDetail_df = spark.read.csv(base_path + "Sales SalesOrderDetail.csv", header=True)
salesTerritory_df = spark.read.csv(base_path + "Sales SalesTerritory.csv", header=True)

sales_details_df = (
    customer_df.join(salesOrderHeader_df, "CustomerID", "left")
    .join(salesOrderDetail_df, "SalesOrderID", "left")
    .join(salesTerritory_df, "TerritoryID", "left")
)

# rename columns : CustomerID -> CustomerKey, Group -> Region
sales_details_df = (
    sales_details_df.withColumnRenamed("CustomerID", "CustomerKey")
    .withColumnRenamed("Group", "Region")
    .withColumnRenamed("PersonID", "BusinessEntityID")
)

sales_details_df = sales_details_df.select(required_cols)

In [31]:
from pyspark.sql.functions import coalesce

# Base path to csv files
base_path = "../raw-files/Production Data/"

# only keep the columns we need for the production data
required_cols = ["ProductID", "ProductCategoryName", "Model"]

# Read required files
product_df = spark.read.csv(base_path + "Production Product.csv", header=True)
productSubcategory_df = spark.read.csv(base_path + "Production ProductSubcategory.csv", header=True)
productCategory_df = spark.read.csv(base_path + "Production ProductCategory.csv", header=True)
productModel_df = spark.read.csv(base_path + "Production ProductModel.csv", header=True)


product_details_df = product_df.join(productSubcategory_df, "ProductSubCategoryID", "left")

# rename column to SubCategoryName
product_details_df = product_details_df.withColumnRenamed("Name", "SubCategoryName")

product_details_df = product_details_df.join(productCategory_df, "ProductCategoryID", "left")

# rename name column to ProductCategoryName
product_details_df = product_details_df.withColumnRenamed("Name", "ProductCategoryName")

product_details_df = product_details_df.join(productModel_df, "ProductModelID", "left")

# Add column Model which is a coalesce of Name and ProductCategoryName
product_details_df = product_details_df.withColumn(
    "Model",
    coalesce(product_details_df["Name"], product_details_df["ProductCategoryName"]),
)

product_details_df = product_details_df.select(required_cols)

In [32]:
# save files to csv
person_output_path = "../denormalized-files/person.csv"
sales_output_path ="../denormalized-files/sales.csv"
products_output_path = "../denormalized-files/products.csv"

# Convert Spark DataFrame to pandas DataFrame and save
person_details_df.toPandas().to_csv(person_output_path, index=False, header=True)
sales_details_df.toPandas().to_csv(sales_output_path, index=False, header=True)
product_details_df.toPandas().to_csv(products_output_path, index=False, header=True)

24/06/25 00:14:37 WARN TransportChannelHandler: Exception in connection from /192.168.100.95:63409
java.io.IOException: Operation timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:378)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.ja